In [2]:
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import utils
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split

In [3]:
dataset_train = './dataset/asl_alphabet_train'
dataset_test = './dataset/asl_alphabet_test'
model_save = './model.hdf5'

In [6]:
imgs=[]
labels=[]
list_dir=os.listdir(dataset_train)
for label in range(len(list_dir)):
    print("Loading images form ", list_dir[label])
    for img_name in os.listdir(dataset_train+"/"+list_dir[label]):
        img=cv.imread(dataset_train+"/"+list_dir[label]+"/"+img_name)
        img=cv.resize(img, (48,48))
        imgs.append(img)
        labels.append(label)

Loading images form  A
Loading images form  B
Loading images form  C
Loading images form  D
Loading images form  del
Loading images form  E
Loading images form  F
Loading images form  G
Loading images form  H
Loading images form  I
Loading images form  J
Loading images form  K
Loading images form  L
Loading images form  M
Loading images form  N
Loading images form  nothing
Loading images form  O
Loading images form  P
Loading images form  Q
Loading images form  R
Loading images form  S
Loading images form  space
Loading images form  T
Loading images form  U
Loading images form  V
Loading images form  W
Loading images form  X
Loading images form  Y
Loading images form  Z


In [7]:
imgs=np.array(imgs)
normalisedX=imgs.astype('float32')/255
label_encode=utils.to_categorical(labels)
x_train, x_test, y_train, y_test=train_test_split(normalisedX, label_encode, test_size=0.2)

In [8]:
classes = 29
batch = 32
epochs = 15
learning_rate = 0.001

In [9]:
model = Sequential()

model.add(Conv2D(64, (3, 3), padding='same', input_shape=(48, 48, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), padding='same', input_shape=(48, 48, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(256, (3, 3), padding='same', input_shape=(48, 48, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dense(classes, activation='softmax'))

In [10]:
adam = Adam(lr=learning_rate)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

c:\Users\danie\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 64)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 24, 24, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 128)      0         
 2D)                                                             
                                                        

In [12]:
history = model.fit(x_train, y_train, batch_size=batch, epochs=epochs, validation_split=0.2, shuffle = True, verbose=1)

Epoch 1/15
1740/1740 [==============================] - 69s 36ms/step - loss: 0.7345 - accuracy: 0.7833 - val_loss: 0.2316 - val_accuracy: 0.9176
Epoch 2/15
1740/1740 [==============================] - 59s 34ms/step - loss: 0.1327 - accuracy: 0.9595 - val_loss: 0.5891 - val_accuracy: 0.8594
Epoch 3/15
1740/1740 [==============================] - 59s 34ms/step - loss: 0.0957 - accuracy: 0.9729 - val_loss: 0.7868 - val_accuracy: 0.8566
Epoch 4/15
1740/1740 [==============================] - 59s 34ms/step - loss: 0.0654 - accuracy: 0.9818 - val_loss: 0.0444 - val_accuracy: 0.9896
Epoch 5/15
1740/1740 [==============================] - 59s 34ms/step - loss: 0.0726 - accuracy: 0.9819 - val_loss: 0.5227 - val_accuracy: 0.9012
Epoch 6/15
1740/1740 [==============================] - 59s 34ms/step - loss: 0.0592 - accuracy: 0.9862 - val_loss: 0.0295 - val_accuracy: 0.9921
Epoch 7/15
1740/1740 [==============================] - 59s 34ms/step - loss: 0.0516 - accuracy: 0.9878 - val_loss: 0.1215 -

In [13]:
model.save(model_save)

: 